In [1]:
import pandas as pd
import pyodbc
import nltk
from  nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [2]:
# connection with DataBase server
def  fetch_data_from_sql():
    connect_string=(
     "Driver={Sql Server};"
     "Server=DESKTOP-3FVNPL5\SQLEXPRESS;"
    "Database=Marketing;"
    "Trusted_connection=yes"
  )
    
    conn=pyodbc.connect(connect_string) 
    query="SELECT ReviewID, CustomerID, ProductID, ReviewDate, Rating, ReviewText FROM customer_reviews"
    df=pd.read_sql(query,conn)
    conn.close()
    return df   

In [3]:

df_customer_reviews=fetch_data_from_sql()
df_customer_reviews.head()


C:\Users\fareed\AppData\Local\Temp\ipykernel_1268\4170904966.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query,conn)


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText
0,1,77,18,2023-12-23,3,"Average experience, nothing special."
1,2,80,19,2024-12-25,5,The quality is top-notch.
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper."
4,5,64,2,2023-07-16,3,"Average experience, nothing special."


In [4]:
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\fareed\AppData\Roaming\nltk_data...


True

In [5]:
sia = SentimentIntensityAnalyzer()


In [6]:
# calculate sentiment scores using VADER
def sentiment_score(review):
    # Get the sentiment scores for the review text
    sentiment = sia.polarity_scores(review)
    # Return the compound score, which is a normalized score between -1 (most negative) and 1 (most positive)
    return sentiment['compound']


In [7]:
#  categorize sentiment using both the sentiment score and the review rating
def categorize_sentiment(score, rating):
    # Use both the text sentiment score and the numerical rating to determine sentiment category
    if score > 0.05:  # Positive sentiment score
        if rating >= 4:
            return 'Positive'  # High rating and positive sentiment
        elif rating == 3:
            return 'Mixed Positive'  # Neutral rating but positive sentiment
        else:
            return 'Mixed Negative'  # Low rating but positive sentiment
    elif score < -0.05:  # Negative sentiment score
        if rating <= 2:
            return 'Negative'  # Low rating and negative sentiment
        elif rating == 3:
            return 'Mixed Negative'  # Neutral rating but negative sentiment
        else:
            return 'Mixed Positive'  # High rating but negative sentiment
    else:  # Neutral sentiment score
        if rating >= 4:
            return 'Positive'  # High rating with neutral sentiment
        elif rating <= 2:
            return 'Negative'  # Low rating with neutral sentiment
        else:
            return 'Neutral'  # Neutral rating and neutral sentiment


In [8]:
# Define a function to bucket sentiment scores into text ranges
def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'  # Strongly positive sentiment
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'  # Mildly positive sentiment
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'  # Mildly negative sentiment
    else:
        return '-1.0 to -0.5'  # Strongly negative sentiment


In [9]:
# Apply sentiment analysis to calculate sentiment scores for each review
df_customer_reviews['SentimentScore'] = df_customer_reviews['ReviewText'].apply(sentiment_score)


In [10]:
#check the data after adding SentimentScore column
df_customer_reviews.head()


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089


In [11]:
# Apply sentiment categorization using both text and rating
df_customer_reviews['SentimentCategory'] = df_customer_reviews.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']), axis=1)


In [12]:
#check the data after adding SentimentCategory column
df_customer_reviews.head()


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative


In [13]:
# Apply sentiment bucketing to categorize scores into defined ranges
df_customer_reviews['SentimentBucket'] = df_customer_reviews['SentimentScore'].apply(sentiment_bucket)


In [14]:
df_customer_reviews.head()


,ReviewID,CustomerID,ProductID,ReviewDate,Rating,ReviewText,SentimentScore,SentimentCategory,SentimentBucket
0,1,77,18,2023-12-23,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0
1,2,80,19,2024-12-25,5,The quality is top-notch.,0.0000,Positive,0.0 to 0.49
2,3,50,13,2025-01-26,4,Five stars for the quick delivery.,0.0000,Positive,0.0 to 0.49
3,4,78,15,2025-04-21,3,"Good quality, but could be cheaper.",0.2382,Mixed Positive,0.0 to 0.49
4,5,64,2,2023-07-16,3,"Average experience, nothing special.",-0.3089,Mixed Negative,-0.49 to 0.0


In [15]:

# Save the DataFrame with sentiment scores, categories, and buckets to a new CSV file
df_customer_reviews.to_csv('customer_reviews_with_sentiment.csv', index = False)
